In [14]:
import pandas as pd
from rapidfuzz import process, fuzz
import json
from collections import defaultdict
import re

In [2]:
df = pd.read_excel('cleaned_d7ee7_data.xlsx')

In [3]:
df.head(5)

,Title,Description,Channel,URL,Views,Likes,Comments,Duration,Duration_seconds
0,ابن مجدي عبد الغني (Epigenetics),مجدي عبد الغني أفحم الناس بضربة الجزاء لكن في ...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=shkThcSCWqM,617313,22053,780,PT4M28S,268
1,ابن مجدي عبد الغني (Epigenetics),مجدي عبد الغني أفحم الناس بضربة الجزاء لكن في ...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=shkThcSCWqM,617313,22053,780,PT4M28S,268
2,إنت جامد,"لمتابعة ""الدحيح"" \nhttp://ajar.io/ryr6j\n\nالد...",AJ+ كبريت,https://www.youtube.com/watch?v=TG3vHrlgczM,1627517,61436,1720,PT4M29S,269
3,ماذا قال عن الأب والأم والعلاقات عن بعد؟,ماذا قال الدحيح عن الأب والأم والعلاقات عن بعد...,New Media Academy Life,https://www.youtube.com/watch?v=UXbGqkCE7Qw,96514,2865,100,PT4M44S,284
4,ليه ما بنتكلمش زي الاجانب؟ (Speech Perception),لمشاهدة الحلقة الأولي من الكورة مع السلامة - ا...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=z9VlaSRmcWE,2232645,105597,2060,PT4M46S,286


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618 entries, 0 to 617
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Title             618 non-null    object
 1   Description       618 non-null    object
 2   Channel           618 non-null    object
 3   URL               618 non-null    object
 4   Views             618 non-null    int64 
 5   Likes             618 non-null    int64 
 6   Comments          618 non-null    int64 
 7   Duration          618 non-null    object
 8   Duration_seconds  618 non-null    int64 
dtypes: int64(4), object(5)
memory usage: 43.6+ KB


In [5]:
data = df
data = data.drop_duplicates(subset='URL', keep='first').reset_index(drop=True)

In [6]:
with open("episode_titles.json", "r", encoding="utf-8") as f:
    episode_titles = json.load(f)

In [7]:
title_to_category = {}
for category, titles in episode_titles.items():
    for title in titles:
        title_to_category[title.strip()] = category

In [8]:
def match_title_to_category(title, choices, threshold=85):
    match, score, _ = process.extractOne(title, choices, scorer=fuzz.token_sort_ratio)
    if score >= threshold:
        return title_to_category[match]
    else:
        return "غير مصنف"

In [9]:
data['Category'] = data['Title'].apply(lambda x: match_title_to_category(x, list(title_to_category.keys())))

In [10]:
data.head(5)

,Title,Description,Channel,URL,Views,Likes,Comments,Duration,Duration_seconds,Category
0,ابن مجدي عبد الغني (Epigenetics),مجدي عبد الغني أفحم الناس بضربة الجزاء لكن في ...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=shkThcSCWqM,617313,22053,780,PT4M28S,268,غير مصنف
1,إنت جامد,"لمتابعة ""الدحيح"" \nhttp://ajar.io/ryr6j\n\nالد...",AJ+ كبريت,https://www.youtube.com/watch?v=TG3vHrlgczM,1627517,61436,1720,PT4M29S,269,الفيزياء
2,ماذا قال عن الأب والأم والعلاقات عن بعد؟,ماذا قال الدحيح عن الأب والأم والعلاقات عن بعد...,New Media Academy Life,https://www.youtube.com/watch?v=UXbGqkCE7Qw,96514,2865,100,PT4M44S,284,غير مصنف
3,ليه ما بنتكلمش زي الاجانب؟ (Speech Perception),لمشاهدة الحلقة الأولي من الكورة مع السلامة - ا...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=z9VlaSRmcWE,2232645,105597,2060,PT4M46S,286,غير مصنف
4,الطويل الزعزوع (Heightism),الطول هيبة والقصر خيبة ! \nhttps://www.patreon...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=TG7xcl1dMnM,378430,12189,330,PT4M51S,291,غير مصنف


In [11]:
def clean_description(text):
    if pd.isnull(text):
        return ""
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r'\b[a-zA-Z]+\b', '', text)
    text = re.sub(r'[^\w\sء-ي]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [15]:
category_examples = defaultdict(list)

for _, row in data.iterrows():
    category = row["Category"]
    desc = clean_description(row.get("Description", ""))
    if category and desc:
        category_examples[category].append(desc)

In [20]:
rows = []
for category, examples in category_examples.items():
    if category != 'غير مصنف':
        for desc in examples:
            rows.append({"Category": category, "Example Description": desc})

examples_df = pd.DataFrame(rows)
examples_df.head()

,Category,Example Description
0,الفيزياء,لمتابعة الدحيح الدحيح إنت جامد ياللي بقالكو كت...
1,الفيزياء,لمتابعة الدحيح الدحيح الثقب الأسود ايه الممر ا...
2,الفيزياء,لمتابعة الدحيح الدحيح تحت الهدوم لو عايزين نشو...
3,الفيزياء,لمشاهدة جميع حلقات الدحيح الدحيح الولا حاجة فن...
4,الفيزياء,لمشاهدة جميع حلقات الدحيح الدحيح المفروض مرفوض...


In [21]:
data[data['Category'] != 'غير مصنف']

,Title,Description,Channel,URL,Views,Likes,Comments,Duration,Duration_seconds,Category
1,إنت جامد,"لمتابعة ""الدحيح"" \nhttp://ajar.io/ryr6j\n\nالد...",AJ+ كبريت,https://www.youtube.com/watch?v=TG3vHrlgczM,1627517,61436,1720,PT4M29S,269,الفيزياء
13,حساسية المياه,"لمتابعة ""الدحيح"" \nhttp://www.youtube.com/c/AJ...",AJ+ كبريت,https://www.youtube.com/watch?v=5dQb3-c_bPo,2507980,119764,3046,PT6M11S,371,الأحياء والبيئة
15,الرأي الآخر,لمشاهدة جميع حلقات الدحيح\nhttp://ajar.io/ueyr...,AJ+ كبريت,https://www.youtube.com/watch?v=EY6f0cdbOWs,1259609,46890,855,PT6M23S,383,علم النفس
17,عنفوان إنسان,"لمتابعة ""الدحيح"" \nhttp://ajar.io/ryr6j\nالدحي...",AJ+ كبريت,https://www.youtube.com/watch?v=61ml6HSa4i4,1461856,52814,988,PT6M31S,391,تاريخ وجغرافيا
27,الثقب الأسود,"لمتابعة ""الدحيح"" \nhttp://ajar.io/ryr6j\nالدحي...",AJ+ كبريت,https://www.youtube.com/watch?v=E-qATMR-l7g,5505577,187006,4580,PT7M14S,434,الفيزياء
...,...,...,...,...,...,...,...,...,...,...
462,بريكست,"لمتابعة ""الدحيح"" \nhttps://goo.gl/sCG87B \n\nش...",AJ+ كبريت,https://www.youtube.com/watch?v=mJVwgFKvADQ,4543629,121174,3708,PT27M47S,1667,سياسة
493,روبين ويليامز,المهرج الحزين الذي أضحك الملايين\n#الدحيح\n\nم...,AJ+ كبريت,https://www.youtube.com/watch?v=hKDxUuSfw9Q,4714717,139325,3639,PT29M20S,1760,سيرة ذاتية
520,أخطر فركشة في التاريخ,"لمتابعة ""الدحيح"" \nhttp://www.youtube.com/c/AJ...",AJ+ كبريت,https://www.youtube.com/watch?v=uhAg97NqLuI,6539749,156899,4409,PT31M30S,1890,سياسة
571,الحرب العالمية الثانية,"لمتابعة ""الدحيح"" \nhttp://www.youtube.com/c/AJ...",AJ+ كبريت,https://www.youtube.com/watch?v=dEiS_WpFuc0,23855609,596126,18594,PT39M14S,2354,تاريخ وجغرافيا
